# ADMISSIONS & OCCUPANCY

## Functions

In [ ]:
from cuhvid.admissions import get_admissions
from cuhvid.bed_occupancy import get_bed_occupancy
from cuhvid.triggers import get_rolling_mean
from cuhvid.wards import get_ward_beds

## Initial parameter values

In [ ]:
_init_vals = dict(
    alpha    = 6.57,
    loc      = 0,
    scale    = 6.56,
    peak_adm = 12,
    sd_coeff = 0.3,
    sd_const = 0.6,
    LoS_GIM  = 9,    # ppt 9    , best 9
    LoS_ICU  = 18,   # ppt 23   , best 18
    frac_ICU = 0.19, # ppt 0.25 , best 0.19
    frac_GIM_amber = 0.50,
    LoS_amber = 1,
    frac_amber_positive = 0.20,
    no_days = 5,
    center = False,
    admissions_trigger = 6,
    net_intake_trigger = 0,
    free_beds_trigger = 20,
    ward_changeup_time=1,
    ward_changedown_time=10,
)

## Data import

In [ ]:
import pandas as pd
import numpy as np

path = '../Data/RAWDATA_Covid_Admissions.xlsx'
df_act = pd.read_excel(path, converters={
    'Date': pd.to_datetime,
    'Admissions': np.float64
})

## Plotting

In [ ]:
from ipywidgets import widgets, Layout
import plotly.graph_objects as go

# ------------------------------------------------------------------------------
# Widgets
# ------------------------------------------------------------------------------

### ..... Admissions .....
reset = widgets.Button(
    description='Reset',
    button_style='warning',
    tooltip='Reset sliders to default values',
    icon='refresh'
)

regen = widgets.Button(
    description='Randomize',
    button_style='',
    tooltip='Regenerate random series keeping all slider values',
    icon='refresh'
)

alpha = widgets.FloatSlider(
    value=_init_vals['alpha'],
    min=0.1,
    max=10,
    step=0.05,
    description='Shape:',
    continuous_update=False,
    readout=True,
    readout_format='.2f',
)

loc = widgets.FloatSlider(
    value=_init_vals['loc'],
    min=-10,
    max=10,
    step=0.1,
    description='Shift:',
    continuous_update=False,
    readout=True,
    readout_format='.1f',
)

scale = widgets.FloatSlider(
    value=_init_vals['scale'],
    min=0.1,
    max=10,
    step=0.01,
    description='Scale:',
    continuous_update=False,
    readout=True,
    readout_format='.2f',
)

peak_adm = widgets.IntSlider(
    value=_init_vals['peak_adm'],
    min=1,
    max=50,
    step=1,
    description='Peak adm.:',
    continuous_update=False,
)

### ..... Variability .....
sd_coeff = widgets.FloatSlider(
    value=_init_vals['sd_coeff'],
    min=0,
    max=1,
    step=0.01,
    description='σ coeff.:',
    continuous_update=False,
    readout=True,
    readout_format='.2f',
)

sd_const = widgets.FloatSlider(
    value=_init_vals['sd_const'],
    min=0,
    max=1,
    step=0.05,
    description='σ const.:',
    continuous_update=False,
    readout=True,
    readout_format='.2f',
)

sd_options = widgets.ToggleButtons(
    options=['Constant', 'Linear Correlation'],
    description='Variability:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Constant variability', 'Variability linearly related to daily admissions'],
    value='Linear Correlation'
)


### ..... Bed occupancy .....
LoS_GIM = widgets.IntSlider(
    value=_init_vals['LoS_GIM'],
    min=1,
    max=50,
    step=1,
    description='LoS GIM:',
    continuous_update=False,
)

LoS_ICU = widgets.IntSlider(
    value=_init_vals['LoS_ICU'],
    min=1,
    max=50,
    step=1,
    description='LoS ICU:',
    continuous_update=False,
)

frac_ICU = widgets.FloatSlider(
    value=_init_vals['frac_ICU'],
    min=0,
    max=1,
    step=0.01,
    description='% ICU:',
    continuous_update=False,
    readout=True,
    readout_format='.0%',
)

frac_GIM_amber = widgets.FloatSlider(
    value=_init_vals['frac_GIM_amber'],
    min=0,
    max=1,
    step=0.01,
    description='% GIM Amber:',
    continuous_update=False,
    readout=True,
    readout_format='.0%',
)

LoS_amber = widgets.IntSlider(
    value=_init_vals['LoS_amber'],
    min=1,
    max=5,
    step=1,
    description='LoS Amber:',
    continuous_update=False,
)

frac_amber_positive = widgets.FloatSlider(
    value=_init_vals['frac_amber_positive'],
    min=0,
    max=1,
    step=0.01,
    description='% Amber +ive:',
    continuous_update=False,
    readout=True,
    readout_format='.0%',
)


### ..... Triggers .....
no_days = widgets.IntSlider(
    value=_init_vals['no_days'],
    min=1,
    max=10,
    step=1,
    description='No days:',
    continuous_update=False,
)

center = widgets.Checkbox(
    value=_init_vals['center'],
    description='Centre rolling mean',
    disabled=False
)

admissions_trigger = widgets.IntSlider(
    value=_init_vals['admissions_trigger'],
    min=1,
    max=15,
    step=1,
    description='Adm. trig.:',
    continuous_update=False,
)
net_intake_trigger = widgets.IntSlider(
    value=_init_vals['net_intake_trigger'],
    min=-5,
    max=5,
    step=1,
    description='Net. trig.:',
    continuous_update=False,
)
free_beds_trigger = widgets.IntSlider(
    value=_init_vals['free_beds_trigger'],
    min=0,
    max=30,
    step=1,
    description='Beds trig.:',
    continuous_update=False,
)


### ..... Ward changeovers .....
ward_changeup_time = widgets.IntSlider(
    value=_init_vals['ward_changeup_time'],
    min=1,
    max=7,
    step=1,
    description='Up leadtime:',
    continuous_update=False,
)

ward_changedown_time = widgets.IntSlider(
    value=_init_vals['ward_changedown_time'],
    min=1,
    max=21,
    step=1,
    description='Down leadtime:',
    continuous_update=False,
)

# ------------------------------------------------------------------------------
# Layout
# ------------------------------------------------------------------------------
layout = go.Layout(
    title='Covid+ admissions: Daily',
    xaxis={
        'title': 'Day',
        'tickmode': 'linear',
        'tick0': 0,
        'dtick': 7
    },
    yaxis={
        'title': 'No. admissions'
    },
    legend=dict(
        x=0,
        y=-0.5
    )
)

layout2 = go.Layout(
    title='Covid+ admissions: Cumulative',
    xaxis={
        'title': 'Day',
        'tickmode': 'linear',
        'tick0': 0,
        'dtick': 7
    },
    yaxis={
        'title': 'Cumulative no. admissions'
    },
    legend=dict(
        x=0,
        y=-0.5
    )
)

layout3 = go.Layout(
    title='Covid bed occupancy: Overall',
    xaxis={
        'title': 'Day',
        'tickmode': 'linear',
        'tick0': 0,
        'dtick': 7
    },
    yaxis={
        'title': 'No. beds'
    },
    legend=dict(
        x=1,
        y=1
    )
)

layout3a = go.Layout(
    title='Covid bed occupancy: GIM',
    xaxis={
        'title': 'Day',
        'tickmode': 'linear',
        'tick0': 0,
        'dtick': 7
    },
    yaxis={
        'title': 'No. beds/admissions'
    },
    legend=dict(
        x=1,
        y=1
    )
)

layout3b = go.Layout(
    title='Covid bed occupancy: ICU',
    xaxis={
        'title': 'Day',
        'tickmode': 'linear',
        'tick0': 0,
        'dtick': 7
    },
    yaxis={
        'title': 'No. beds/admissions'
    },
    legend=dict(
        x=1,
        y=1
    )
)

layout4 = go.Layout(
    title='Trigger signal: Daily Covid+ admissions',
    xaxis={
        'title': 'Day',
        'tickmode': 'linear',
        'tick0': 0,
        'dtick': 7
    },
    yaxis={
        'title': 'No. admissions'
    },
    legend=dict(
        x=0,
        y=-0.5
    )
)

layout5 = go.Layout(
    title='Trigger signal: Daily net Covid+ patient intake',
    xaxis={
        'title': 'Day',
        'tickmode': 'linear',
        'tick0': 0,
        'dtick': 7
    },
    yaxis={
        'title': 'Net patient intake'
    },
    legend=dict(
        x=0,
        y=-0.5
    )
)

layout5a = go.Layout(
    title='Trigger signal: Open Covid beds',
    xaxis={
        'title': 'Day',
        'tickmode': 'linear',
        'tick0': 0,
        'dtick': 7
    },
    yaxis={
        'title': 'No. free beds'
    },
    legend=dict(
        x=0,
        y=-0.5
    )
)

layout6 = go.Layout(
    title='Covid bed configuration and occupancy: Red GIM wards',
    xaxis={
        'title': 'Day',
        'tickmode': 'linear',
        'tick0': 0,
        'dtick': 7
    },
    yaxis={
        'title': 'No. beds/patients'
    },
    legend=dict(
        x=0,
        y=-0.5
    )
)

layout7 = go.Layout(
    title='Covid bed configuration and occupancy: Amber GIM wards',
    xaxis={
        'title': 'Day',
        'tickmode': 'linear',
        'tick0': 0,
        'dtick': 7
    },
    yaxis={
        'title': 'No. beds'
    },
    legend=dict(
        x=0,
        y=-0.5
    )
)


# ------------------------------------------------------------------------------
# Functions to link widget values to dataframe functions
# ------------------------------------------------------------------------------
def get_admissions_params():
    """
    Returns dictionary of values of parameters for get_admissions()
    
    e.g. usage: 
        get_admissions(**get_admissions_params())
    """
    params = dict(alpha=alpha.value,
                  loc=loc.value,
                  scale=scale.value,
                  peak_adm=peak_adm.value,
                  sd_coeff=sd_coeff.value,
                  sd_const=sd_const.value)
    return params

def get_occupancy_params():
    """
    Returns dictionary of values of parameters for get_bed_occupancy()
    
    e.g. usage: 
        get_bed_occupancy(df, **get_occupancy_params())
    """
    params = dict(LoS_GIM=LoS_GIM.value,
                  LoS_ICU=LoS_ICU.value,
                  frac_ICU=frac_ICU.value,
                  frac_GIM_amber=frac_GIM_amber.value,
                  LoS_amber=LoS_amber.value,
                  frac_amber_positive=frac_amber_positive.value)
    return params

def get_rolling_params():
    """
    Returns dictionary of values of parameters for get_rolling_mean()
    
    e.g. usage: 
        get_rolling_mean(df, columns, **get_rolling_params())
    """
    params = dict(no_days=no_days.value,
                  center=center.value)
    return params


# ------------------------------------------------------------------------------
# Populate dataframe
# ------------------------------------------------------------------------------
df_gen = get_admissions(**get_admissions_params())
df_gen = get_bed_occupancy(df_gen, **get_occupancy_params())

rolling_mean_columns = ['y_gen', 'net_intake_gen']
df_gen = get_rolling_mean(df_gen, rolling_mean_columns, **get_rolling_params())

df_gen = get_ward_beds(df_gen)


# ------------------------------------------------------------------------------
# Set up plot traces
# ------------------------------------------------------------------------------

## --------------- Daily data ---------------
data = []
data.append(go.Scatter(
    name = 'Curve fit and variance',
    x = df_gen.index,
    y = df_gen.y_lower,
    mode='lines',
    line=dict(width=0),
    showlegend=False
))
data.append(go.Scatter(
    name = 'Curve fit and variance',
    x = df_gen.index,
    y = df_gen.y,
    mode='lines',
    line=dict(color='rgb(0, 176, 246)'),
    fill='tonexty',
    fillcolor='rgba(0, 176, 246, 0.3)',
))
data.append(go.Scatter(
    name = 'Curve fit and variance',
    x = df_gen.index,
    y = df_gen.y_upper,
    mode='lines',
    line=dict(width=0),
    fill='tonexty',
    fillcolor='rgba(0, 176, 246, 0.3)',
    showlegend=False
))
data.append(go.Scatter(
    name = 'Generated series',
    x = df_gen.index,
    y = df_gen.y_gen,
    mode='lines',
    line=dict(color='rgb(166, 139, 165)')
))
data.append(go.Scatter(
    name = 'Admissions (first wave)',
    x = df_gen.index,
    y = df_act.Admissions,
    mode='lines',
    line=dict(color='rgb(98, 76, 171)', dash='dot'),
))

## --------------- Cumulative data ---------------
data2 = []
data2.append(go.Scatter(
    name = 'Curve fit and variance',
    x = df_gen.index,
    y = df_gen.y.cumsum(),
    mode='lines',
    line=dict(color='rgb(0, 176, 246)'),
))
data2.append(go.Scatter(
    name = 'Generated series',
    x = df_gen.index,
    y = df_gen.y_gen.cumsum(),
    mode='lines',
    line=dict(color='rgb(166, 139, 165)')
))
data2.append(go.Scatter(
    name = 'Admissions (first wave)',
    x = df_gen.index,
    y = df_act.Admissions.cumsum().values,
    mode='lines',
    line=dict(color='rgb(98, 76, 171)', dash='dot'),
))

## --------------- Bed occupancy data ---------------
data3 = []
data3.append(go.Scatter(
    name = 'Generated GIM',
    x = df_gen.index,
    y = df_gen.GIM_gen,
    mode='lines',
    line=dict(color='rgb(166, 139, 165)')
))
data3.append(go.Scatter(
    name = 'Actual GIM',
    x = df_gen.index,
    y = df_act['GIM bed occupancy'],
    mode='lines',
    line=dict(color='rgb(98, 76, 171)')
))
data3.append(go.Scatter(
    name = 'Fitted GIM',
    x = df_gen.index,
    y = df_gen.GIM,
    mode='lines',
    line=dict(color='rgb(0, 176, 246)'),
))
data3.append(go.Scatter(
    name = 'Generated ICU',
    x = df_gen.index,
    y = df_gen.ICU_gen,
    mode='lines',
    line=dict(color='rgb(166, 139, 165)', dash='dot')
))
data3.append(go.Scatter(
    name = 'Actual ICU',
    x = df_gen.index,
    y = df_act['ICU bed occupancy'],
    mode='lines',
    line=dict(color='rgb(98, 76, 171)', dash='dot'),
))
data3.append(go.Scatter(
    name = 'Fitted ICU',
    x = df_gen.index,
    y = df_gen.ICU,
    mode='lines',
    line=dict(color='rgb(0, 176, 246)', dash='dot'),
))

### ..... Amber bed occupancy .....
data3a = []
data3a.append(go.Scatter(
    name = 'GIM total bed occupancy',
    x = df_gen.index,
    y = df_gen.GIM_gen,
    mode='lines',
    line=dict(color='#030303')
))
data3a.append(go.Scatter(
    name = 'GIM red bed occupancy',
    x = df_gen.index,
    y = df_gen.GIM_R_gen,
    mode='lines',
    line=dict(color='#ff0000')
))
data3a.append(go.Scatter(
    name = 'GIM amber bed occupancy',
    x = df_gen.index,
    y = df_gen.GIM_A_gen,
    mode='lines',
    line=dict(color='#ffbf00')
))
data3a.append(go.Scatter(
    name = 'Total Admissions',
    x = df_gen.index,
    y = df_gen.y_gen,
    mode='lines',
    line=dict(color='#4e7acc', dash='dot')
))

### ..... Red bed occupancy .....
data3b = []
data3b.append(go.Scatter(
    name = 'ICU bed occupancy',
    x = df_gen.index,
    y = df_gen.ICU_gen,
    mode='lines',
    line=dict(color='#030303')
))
data3b.append(go.Scatter(
    name = 'Total Admissions',
    x = df_gen.index,
    y = df_gen.y_gen,
    mode='lines',
    line=dict(color='#4e7acc', dash='dot')
))

## --------------- Admissions trigger ---------------
data4 = []
data4.append(
    go.Scatter(
        name = 'Admissions - Daily',
        x = df_gen.index,
        y = df_gen.y_gen,
        mode = 'lines',
        opacity = 0.5
    )
)
data4.append(
    go.Scatter(
        name = f'Admissions - ({no_days.value} day rolling mean)',
        x = df_gen.index,
        y = df_gen.y_gen_rm
    )
)
data4.append(
    go.Scatter(
        name = 'Admissions trigger',
        x = [df_gen.index.min(), df_gen.index.max()],
        y = [admissions_trigger.value, admissions_trigger.value],
        mode = 'lines',
        line=dict(color='#0e0e0e')
    )
)

## --------------- Net intake trigger ---------------
data5 = []
data5.append(
    go.Scatter(
        name = 'Net intake - Daily',
        x = df_gen.index,
        y = df_gen.net_intake_gen,
        mode = 'lines',
        opacity = 0.5
    )
)
data5.append(
    go.Scatter(
        name = f'Net intake - ({no_days.value} day rolling mean)',
        x = df_gen.index,
        y = df_gen.net_intake_gen_rm
    )
)
data5.append(
    go.Scatter(
        name = 'Net intake trigger',
        x = [df_gen.index.min(), df_gen.index.max()],
        y = [net_intake_trigger.value, net_intake_trigger.value],
        mode = 'lines',
        line=dict(color='#0e0e0e')
    )
)


## --------------- Available beds ---------------
data5a = []
data5a.append(go.Scatter(
    name = 'GIM red beds available',
    x = df_gen.index,
    y = df_gen.GIM_R_beds_avail,
    mode='lines',
    line=dict(color='#ff0000')
))
data5a.append(go.Scatter(
    name = 'GIM amber beds available',
    x = df_gen.index,
    y = df_gen.GIM_A_beds_avail,
    mode='lines',
    line=dict(color='#ffbf00')
))
data5a.append(go.Scatter(
    name = 'Free beds trigger',
    x = [df_gen.index.min(), df_gen.index.max()],
    y = [free_beds_trigger.value, free_beds_trigger.value],
    mode = 'lines',
    line=dict(color='#0e0e0e')
))

## --------------- Red beds & occupancy ---------------
data6 = []
data6.append(go.Scatter(
    name = 'GIM red bed occupancy',
    x = df_gen.index,
    y = df_gen.GIM_R_gen,
    mode='lines',
    line=dict(color='#ff0000')
))
data6.append(go.Scatter(
    name = 'GIM red bed availability',
    x = df_gen.index,
    y = df_gen.GIM_R_beds,
    mode='lines',
    line=dict(color='#b00000')
))

## --------------- Amber beds & occupancy ---------------
data7 = []
data7.append(go.Scatter(
    name = 'GIM amber bed occupancy',
    x = df_gen.index,
    y = df_gen.GIM_A_gen,
    mode='lines',
    line=dict(color='#ffbf00')
))
data7.append(go.Scatter(
    name = 'GIM amber bed availability',
    x = df_gen.index,
    y = df_gen.GIM_A_beds,
    mode='lines',
    line=dict(color='#ba8b00')
))

# ------------------------------------------------------------------------------
# Plot
# ------------------------------------------------------------------------------
g = go.FigureWidget(data=data, layout=layout)
g2 = go.FigureWidget(data=data2, layout=layout2)
g3 = go.FigureWidget(data=data3, layout=layout3)
g3a = go.FigureWidget(data=data3a, layout=layout3a)
g3b = go.FigureWidget(data=data3b, layout=layout3b)
g4 = go.FigureWidget(data=data4, layout=layout4)
g5 = go.FigureWidget(data=data5, layout=layout5)
g5a = go.FigureWidget(data=data5a, layout=layout5a)
g6 = go.FigureWidget(data=data6, layout=layout6)
g7 = go.FigureWidget(data=data7, layout=layout7)


# ------------------------------------------------------------------------------
# Table
# ------------------------------------------------------------------------------
from cuhvid.wards import get_ward_change_rank
df_ward_rank = get_ward_change_rank()

import plotly.figure_factory as ff
t1 = go.FigureWidget(ff.create_table(df_ward_rank))


# ------------------------------------------------------------------------------
# Responsive functions
# ------------------------------------------------------------------------------

## --------------- Replot on change ---------------
def response(change):
    # ..... Repopulate dataframe .....
    df_gen = get_admissions(**get_admissions_params())
    df_gen = get_bed_occupancy(df_gen, **get_occupancy_params())
    df_gen = get_rolling_mean(df_gen, rolling_mean_columns, **get_rolling_params())
    df_gen = get_ward_beds(df_gen)

    with g.batch_update():
        g.data[0].y = df_gen.y_lower
        g.data[1].y = df_gen.y
        g.data[2].y = df_gen.y_upper
        g.data[3].y = df_gen.y_gen
        
    with g2.batch_update():
        g2.data[0].y = df_gen.y.cumsum()
        g2.data[1].y = df_gen.y_gen.cumsum()
        
    with g3.batch_update():
        g3.data[0].y = df_gen.GIM_gen
        g3.data[2].y = df_gen.GIM
        g3.data[3].y = df_gen.ICU_gen
        g3.data[5].y = df_gen.ICU
    
    with g3a.batch_update():
        g3a.data[0].y = df_gen.GIM_gen
        g3a.data[1].y = df_gen.GIM_R_gen
        g3a.data[2].y = df_gen.GIM_A_gen
        g3a.data[3].y = df_gen.y_gen
        
    with g3b.batch_update():
        g3b.data[0].y = df_gen.ICU_gen
        g3b.data[1].y = df_gen.y_gen
    
    with g4.batch_update():
        g4.data[0].y = df_gen.y_gen
        g4.data[1].y = df_gen.y_gen_rm
        g4.data[2].y = [admissions_trigger.value, admissions_trigger.value]
        
    with g5.batch_update():
        g5.data[0].y = df_gen.net_intake_gen
        g5.data[1].y = df_gen.net_intake_gen_rm
        g5.data[2].y = [net_intake_trigger.value, net_intake_trigger.value]
        
    with g5a.batch_update():
        g5a.data[0].y = df_gen.GIM_R_beds_avail
        g5a.data[1].y = df_gen.GIM_A_beds_avail
        g5a.data[2].y = [free_beds_trigger.value, free_beds_trigger.value]
        
    with g6.batch_update():
        g6.data[0].y = df_gen.GIM_R_gen
        g6.data[1].y = df_gen.GIM_R_beds
    
    with g7.batch_update():
        g7.data[0].y = df_gen.GIM_A_gen
        g7.data[1].y = df_gen.GIM_A_beds
    
    return

## --------------- Reset to default values ---------------
def reset_values(change):
    alpha.value = _init_vals['alpha']
    loc.value = _init_vals['loc']
    scale.value = _init_vals['scale']
    peak_adm.value = _init_vals['peak_adm']
    sd_coeff.value = _init_vals['sd_coeff']
    sd_const.value = _init_vals['sd_const']
    LoS_GIM.value = _init_vals['LoS_GIM']
    LoS_ICU.value = _init_vals['LoS_ICU']
    frac_ICU.value = _init_vals['frac_ICU']
    LoS_amber.value = _init_vals['LoS_amber']
    frac_GIM_amber = _init_vals['frac_GIM_amber']
    no_days.value = _init_vals['no_days']
    center.value = _init_vals['center']
    admissions_trigger.value = _init_vals['admissions_trigger']
    net_intake_trigger.value = _init_vals['net_intake_trigger']
    free_beds_trigger.value = _init_vals['free_beds_trigger']
    ward_changeup_time.value = _init_vals['ward_changeup_time']
    ward_changedown_time.value = _init_vals['ward_changedown_time']
    
    response(None)
    return

## --------------- Variability options ---------------
def variability(change):
    if change['new'] == 'Constant':
        sd_coeff.value = 0
        sd_const.value = 0.75
        sd_coeff.disabled = True
    elif change['new'] == 'Linear Correlation':
        sd_coeff.value = _init_vals['sd_coeff']
        sd_const.value = _init_vals['sd_const']
        sd_coeff.disabled = False
    return


# ------------------------------------------------------------------------------
# Link widgets to functions
# ------------------------------------------------------------------------------
reset.on_click(reset_values)

regen.on_click(response)

sd_options.observe(variability, names='value')

alpha.observe(response, names='value')
loc.observe(response, names='value')
scale.observe(response, names='value')
peak_adm.observe(response, names='value')
sd_coeff.observe(response, names='value')
sd_const.observe(response, names='value')

LoS_GIM.observe(response, names='value')
LoS_ICU.observe(response, names='value')
frac_ICU.observe(response, names='value')
frac_GIM_amber.observe(response, names='value')
LoS_amber.observe(response, names='value')
frac_amber_positive.observe(response, names='value')

no_days.observe(response, names='value')
center.observe(response, names='value')
admissions_trigger.observe(response, names='value')
net_intake_trigger.observe(response, names='value')
free_beds_trigger.observe(response, names='value')

ward_changeup_time.observe(response, names='value')
ward_changedown_time.observe(response, names='value')


# ------------------------------------------------------------------------------
# Display widgets & plots
# ------------------------------------------------------------------------------

## --------------- Widgets ---------------
### ..... Admissions .....
adm_desc = widgets.HTML('''<p>
    Generate a covid patient admissions signal.
    </p>''', 
    layout=Layout(max_width='600px')
)
gamma_control = widgets.VBox([
    widgets.HTML(value='<h2>Curve parameters</h2>'),
    alpha,
    loc,
    scale,
    peak_adm,
])
variability_control = widgets.VBox([
    widgets.HTML(value='<h2>Variability parameters</h2>'),
    sd_coeff,
    sd_const,
    sd_options
])
button_control = widgets.VBox([
    reset,
    regen
])

admissions_control = widgets.HBox([
    gamma_control,
    variability_control,
    button_control
], layout=Layout(justify_content='space-around'))
admissions_plots = widgets.HBox([
    g,
    g2
], layout=Layout(justify_content='space-around'))

admissions = widgets.VBox([
    widgets.HTML(value='<h1>Covid+ Patient Admissions</h1>'),
    adm_desc,
    admissions_control,
    admissions_plots
])

### ..... Bed occupancy .....
occ_desc = widgets.HTML('''<p>
    Use the generated covid patient admissions signal to determine 
    the required bed occupancy demand.
    </p>''', 
    layout=Layout(max_width='600px')
)
occ_control = widgets.VBox([
    widgets.HTML(value='<h2>Red bed occupancy</h2>'),
    LoS_GIM,
    LoS_ICU,
    frac_ICU,
    widgets.HTML(value='<h2>Amber bed occupancy</h2>'),
    frac_GIM_amber,
    LoS_amber,
    frac_amber_positive
], layout=Layout(align_self='center'))

occupancy = widgets.VBox([
    widgets.HTML(value='<h1>Covid Ward Bed Occupancy</h1>'),
    occ_desc,
    widgets.HBox([
        occ_control,
        g3
    ], layout=Layout(justify_content='center')),
    widgets.HBox([
        g3a,
        g3b
    ], layout=Layout(justify_content='space-around'))
])

### ..... Triggers .....
trigger_desc = widgets.HTML('''<p>
    Plot and evaluate trigger signals to determine when opening/closing 
    of wards would occur.
    </p>''', 
    layout=Layout(max_width='600px')
)
trigger_control = widgets.VBox([
    widgets.HTML(value='<h2>Trigger signal smoothing parameters</h2>'),
    no_days,
    center
])
trigger_plots = widgets.HBox([
    widgets.VBox([admissions_trigger, g4]),
    widgets.VBox([net_intake_trigger, g5]),
    widgets.VBox([free_beds_trigger, g5a])
], layout=Layout(justify_content='space-around'))

triggers = widgets.VBox([
    widgets.HTML(value='<h1>Covid Ward Open/Close Trigger Signals</h1>'),
    trigger_desc,
    trigger_control,
    trigger_plots
])

### ..... Ward configuration .....
ward_desc = widgets.HTML('''<p>
    Generate an order to open/close wards and simulate its use in line 
    with the trigger signals.
    </p>''', 
    layout=Layout(max_width='600px')
)

ward_control = widgets.VBox([
    widgets.HTML(value='<h2>Ward changeover times</h2>'),
    ward_changeup_time,
    ward_changedown_time
])
wards = widgets.VBox([
    widgets.HTML(value='<h1>Hospital Covid Ward Configuration</h1>'),
    ward_desc,
    widgets.HTML(value='<h2>Ward changeover order</h2>'),
    widgets.HBox([
        ward_control,
        t1
    ], layout=Layout(justify_content='space-around')),
    widgets.HBox([
        g6,
        g7
    ], layout=Layout(justify_content='space-around'))
])

headnote = widgets.HTML(
    "<h1 style='text-align: center'>Configuration planning — Interactive tactical tool</h1>"
)
footnote = widgets.HTML(
    value='''<p style='text-align: center; font-style: italic'>
    Made by O. C. Lambson (University of Cambridge Institute for Manufacturing) 
    for Cambridge University Hospitals NHS Foundation Trust. This tool was shaped
    significantly by contributions from J. S. Srai and E. Settanni. 
    <br/>
    July 2020
    </p>'''
)
## --------------- Display ---------------
widgets.VBox([
    headnote,
    admissions,
    widgets.HTML(value='<hr>'),
    occupancy,
    widgets.HTML(value='<hr>'),
    triggers,
    widgets.HTML(value='<hr>'),
    wards,
    widgets.HTML(value='<hr>'),
    footnote
], layout=Layout(width='90vw', margin='auto'))

# \_\_Debugging\_\_

## Master dataframe output

In [ ]:
from IPython.display import display
display(df_gen[35:42])

In [ ]:
# from cuhvid.wards import get_ward_beds
# get_ward_beds(df_gen)